In [4]:
import pandas as pd
import os
import math

In [2]:
fte = pd.read_csv(os.path.expanduser('~/Downloads/spi_matches.csv'))

In [ ]:
fte.columns

## Put two things online for every league
1. Table
2. Transformed aggregate version w/ one team like cte on sql query

In [ ]:
def mvp(df, date, home_team, away_team, home_goals, away_goals, home_xg, away_xg, home_nsxg, away_nxsg, ):
    #home
    home = df.groupby(home_team).apply(
        lambda x: (x[home_goals] > x[away_goals]).sum() * 3 + (x[home_goals] == x[away_goals]).sum()).reset_index(name='points')
    pt = pd.pivot_table(df, values = [home_goals, away_goals], index = home_team, aggfunc=sum)
    home = home.merge(pt.reset_index(), on = home_team)
    pt = pd.pivot_table(df, values = date, index = home_team, aggfunc=pd.Series.nunique)
    home = home.merge(pt.reset_index(), on = home_team)
    home = home.rename(columns = {home_team:'team', away_goals:'conceded', home_goals: 'scored', date: 'played'})
    if results == 'home':
        t = home
#away
    away = df.groupby(away_team).apply(
        lambda x: (x[away_goals] > x[home_goals]).sum() * 3 + (x[home_goals] == x[away_goals]).sum()).reset_index(name='points')
    pt = pd.pivot_table(df, values = [home_goals, away_goals], index = away_team, aggfunc=sum)
    away = away.merge(pt.reset_index(), on = away_team)
    pt = pd.pivot_table(df, values = date, index = away_team, aggfunc=pd.Series.nunique)
    away = away.merge(pt.reset_index(), on = away_team)
    away = away.rename(columns = {away_team:'team', away_goals:'scored', home_goals: 'conceded', date: 'played'})
    if results == 'away':
        t = away
#merge
    t = pd.concat([home, away], ignore_index=True)
    t = t.groupby('team').sum()
    t['goal_difference'] = t['scored'] - t['conceded']
    return t

In [20]:
def fte_last_season(path, league_dict):
    df = pd.read_csv(path)
    for key in league_dict:
        q = ''.join(['league_id == ', league_dict[key]])
        league = df.query(q).reset_index()
        num_teams = len(league.loc[:50, 'team1'].unique())
        i = math.floor(league.shape[0]/(num_teams * (num_teams - 1))) - 1
        return league.loc[((num_teams * (num_teams - 1)) * i):,:]

In [23]:
path = os.path.expanduser('~/Downloads/spi_matches.csv')
league_dict = {'england':'2411', 'spain':'1869', 'italy':'1854','germany':'1845', 'france':'1843'}
fte = fte_last_season(path, league_dict)